<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


###IBAN Generator###

In [0]:
# IBAN Generator

import datetime
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)
print (eiban)





LC7200322800994985921


###Payment Generator###

In [174]:
# Erstellung von Transaktionsdaten 

from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# Bankbeziehungsdaten Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','IBAN','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 


# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)

# Betrugseinstufung 
# Kein Fraud ist immer 0
fraud_ids = [[1,'External Caused Fraud (e.g. Trojans)'],[2,'Internal Caused Fraud (e.g. Employee)']]
df_fraud = pd.DataFrame(fraud_ids, columns=['Fraud ID', 'Description'])


# Bankkunden Stammdaten - Datenquellen Github
custdb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/cutomer_base_data.csv'

cust_data = pd.read_csv(custdb_url, delimiter=';')
cust_data.columns=['anrede','vorname','nachname','strasse','nr','plz','ort',
                   'kanton','sprachregion',	'nation',	'sexcode','mobile_nr',	
                   'vertrag_nr','konto_nr','auth_device']

# Importierte Kundendaten bereinigen
cust_data = cust_data.fillna(0)
cust_data_dtype = {'plz':int,'mobile_nr':int,'vertrag_nr':int}
cust_data = cust_data.astype(cust_data_dtype)

# Erstellung der Zahlungs-Basisinformationen 

#start_date = datetime(2018,1,1,8,0,0)
trx_data = pd.DataFrame(columns=['timestamp','paym_id','send_bc_nr','amount',
                                 'rcv_bc_nr','rcv_bc_iban','rcv_bc_country','rcv_bc_code',
                                 'valuta_date','cust_vertrag_nr','cust_konto_nr',
                                 'cust_vname','cust_nname','cust_plz','cust_ort',
                                'cust_kanton','cust_sprachregion','cust_auth_device',
                                'fraud_id'])

for i in range(100):
    trx_data.loc[i,['timestamp']] = timestamps = pd.Timestamp(datetime(2018,1,1,8,0,0)+timedelta(seconds=30*i)).strftime("%Y.%m.%d %H:%M")
    trx_data.loc[i,['paym_id']] =  np.int64(np.random.randint(1000000000000,2000000000000,1))
    trx_data.loc[i,['send_bc_nr']] =  'CHBANK71XXX'
    trx_data.loc[i,['amount']] = np.int64(np.random.randint(100,100000,1)).astype(str)
    trx_data.loc[i,['rcv_bc_nr']] = bc_bank = np.random.choice(np.ravel(iban_cc['Bank']).astype(str))
    trx_data.loc[i,['rcv_bc_iban']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']])
    trx_data.loc[i,['rcv_bc_country']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Country']])
    trx_data.loc[i,['rcv_bc_code']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Code']])
    timestamps = datetime.strptime(timestamps,"%Y.%m.%d %H:%M")
    trx_data.loc[i,['valuta_date']] = valuta = timestamps + timedelta(hours=12*i)
    trx_data.loc[i,['cust_vertrag_nr']] = cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))
    trx_data.loc[i,['cust_konto_nr']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr']])
    trx_data.loc[i,['cust_vname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['vorname']])
    trx_data.loc[i,['cust_nname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['nachname']])
    trx_data.loc[i,['cust_plz']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['plz']])
    trx_data.loc[i,['cust_ort']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['ort']])
    trx_data.loc[i,['cust_kanton']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['kanton']])
    trx_data.loc[i,['cust_sprachregion']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['sprachregion']])
    trx_data.loc[i,['cust_auth_device']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['auth_device']])
    trx_data.loc[i,['fraud_id']] = 0
    

# Von allen durchgeführten Zahlungen hat das Fraud-Detection System 0.5% Betrugszahlungen erkannt. 
trx_data_fraud = trx_data.sample(frac=0.05)
#trx_data_fraud.fraud_id = ''


for x in trx_data_fraud['fraud_id']:
      test = np.random.choice(20,1)
      for i in range(len(trx_data_fraud)):
        trx_data_fraud.replace([0],test,inplace=True)
      #trx_data_fraud.loc[x,['fraud_id']] = np.random.choice(2,1)

trx_data_fraud.head()
  
  





,timestamp,paym_id,send_bc_nr,amount,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device,fraud_id
74,2018.01.01 08:37,1449127130583,CHBANK71XXX,62131,LUBANK50XXX,LU020019175546294000,Luxemburg,LU,2018-02-07 08:37:00,70967,CH5300772922055953881,Françoise,Gashi,1219,Châtelaine,GE,fr,ptan,17
27,2018.01.01 08:13,1193009747819,CHBANK71XXX,35631,HUBANK82XXX,HU02116000060000000064247067,Ungarn,HU,2018-01-14 20:13:00,65324,CH5300772714814918250,Mélanie,Martin,1292,Chambésy,GE,fr,mtan,17
10,2018.01.01 08:05,1522201272313,CHBANK71XXX,78374,MTBANK51XXX,MT02VALL22013000000040013752732,Malta,MT,2018-01-06 08:05:00,82730,CH5300772042576124212,Josef,Steiner,6102,Malters,LU,de,ptan,17
15,2018.01.01 08:07,1600293569263,CHBANK71XXX,82292,SCBANK73XXX,SC74NOVH00000021002035257028SCR,Seychellen,SC,2018-01-08 20:07:00,75482,CH5300772501152624645,Sandra,Kopp,6014,Luzern,LU,de,mtan,17
4,2018.01.01 08:02,1846468202326,CHBANK71XXX,80129,AEBANK83XXX,AE020090004001079346500,Vereinigte Arabische Emirate,AE,2018-01-03 08:02:00,68211,CH5300772829737707253,Thomas,Müller,6014,Luzern,LU,de,mtan,17


In [0]:


import pandas as pd
from random import randint

df = pd.DataFrame(columns=['A','B','C','D','E'])

for i in range(5): #add 5 rows of data
    df.loc[i, ['A']] = randint(0,99)
    df.loc[i, ['B']] = randint(0,99)
    df.loc[i, ['C']] = randint(0,99)
    df.loc[i, ['D']] = randint(0,99)
    df.loc[i, ['E']] = randint(0,99)
    
df

,A,B,C,D,E
0,82.0,93.0,1.0,5.0,8.0
1,4.0,34.0,45.0,13.0,88.0
2,87.0,41.0,54.0,41.0,43.0
3,81.0,44.0,98.0,3.0,83.0
4,47.0,52.0,20.0,45.0,60.0
